#### El cliente ha corroborado que podemos manejar RDD's pero el CORE a utilizar será dataframes, nos han solicitado crear algunas funciones para cargar archivos en formato csv y almacenarlos en formato parquet. ¿Por qué en parquet? Hecha un ojo a esto (vendrá en la evaluación final):
#### https://www.databricks.com/glossary/what-is-parquet
#### Cuando termines resuelve cada actividad planteada

#### Actividad 1:
##### TO DO: Crear sparkSession con appName "ejercicio_4", y master "local[*]", almacenar sesion de spark en una variable llamada "spark"

In [33]:
# Declaración de librerias
from pyspark.sql import SparkSession, DataFrame

In [34]:
## TU CODIGO VA EN ESTA CELDA:
# levantar sesion de spark
spark = SparkSession.builder.appName("ejercicio_4").master("local[*]").getOrCreate() # levantar sesión de spark
sc = spark.sparkContext # crear contexto de spark

In [35]:
# NO MODIFICAR CONTENIDO DE ESTA CELDA
assert type(spark) == SparkSession
assert spark.sparkContext.getConf().get("spark.master") == "local[*]"
assert spark.sparkContext.getConf().get("spark.app.name") == "ejercicio_4"

#### Actividad 2:
##### TO DO ->    Crear un método para leer un archivo en formato csv, no inferir el esquema, los archivos si incluyen header, el separador es ","
- ##### retorna ->  dataframe correspondiente a la lectura
- ##### firma ->    def read_csv(path: str) -> DataFrame

In [36]:
## TU CODIGO VA EN ESTA CELDA:
def read_csv(path: str) -> DataFrame:
    return spark.read\
        .option("header","true")\
        .option("delimiter",",")\
        .option("inferSchema","false")\
        .csv(path)

In [37]:
# NO MODIFICAR CONTENIDO DE ESTA CELDA
from pyspark.sql import DataFrame

root_path = "../resources/data/csv/"

movies_df = read_csv(root_path + "movies.csv")
ratings_df = read_csv(root_path + "ratings.csv")
tags_df = read_csv(root_path + "tags.csv")

def schema_to_ddl(df):
    return spark.sparkContext._jvm.org.apache.spark.sql.types.DataType.fromJson(df.schema.json()).toDDL()

assert (type(movies_df) == DataFrame) & (type(ratings_df) == DataFrame) & (type(tags_df) == DataFrame)
assert (movies_df.count() == 86537) & (ratings_df.count() == 33832162) & (tags_df.count() == 2328315)
assert schema_to_ddl(movies_df) == 'movieId STRING,title STRING,genres STRING'
assert schema_to_ddl(ratings_df) == 'userId STRING,movieId STRING,rating STRING,timestamp STRING'
assert schema_to_ddl(tags_df) == 'userId STRING,movieId STRING,tag STRING,timestamp STRING'

#### Actividad 3:
##### TO DO ->    Crear un método para escribir un dataframe con las siguientes caracteristicas:
- ##### formato: parquet
- ##### modo de escritura: overwrite
- ##### firma: def write_parquet(df: DataFrame, path: str):

In [38]:
## TU CODIGO VA EN ESTA CELDA:
def write_parquet(df: DataFrame, path: str):
    return df.write.mode("overwrite").parquet(path)
    

In [39]:
# NO MODIFICAR CONTENIDO DE ESTA CELDA

root_path = "../resources/data/tmp/parquet/04/"

write_parquet(movies_df, root_path + "movies")
write_parquet(ratings_df, root_path + "ratings")
write_parquet(tags_df, root_path + "tags")

_movies_df = spark.read.parquet(root_path + "movies")
_ratings_df = spark.read.parquet(root_path + "ratings")
_tags_df = spark.read.parquet(root_path + "tags")

assert _movies_df.count() == 86537
assert _ratings_df.count() == 33832162
assert _tags_df.count() == 2328315